## Goal - Points to Lines

In [ ]:
import os

import geopandas as gp

from osgeo import ogr

In [ ]:
pntshp = '/mnt/x/pwgis/code24data/demo8gdal/pnt_roads_gdal.shp'
lnhshp = '/mnt/x/pwgis/code24data/results/demo8gdal.shp'

fid_field = 'ofid'
pnt_order = 'cpnt'

In [ ]:
"""
GDAL Drivers Name
"""

def drv_name(_file):
    """
    Return the driver for a given file format
    """
    
    drv = {
        # Vector files
        '.gml'    : 'GML',
        '.shp'    : 'ESRI Shapefile',
        '.json'   : 'GeoJSON',
        '.kml'    : 'KML',
        '.osm'    : 'OSM',
        '.dbf'    : 'ESRI Shapefile',
        '.vct'    : 'Idrisi',
        '.nc'     : 'netCDF',
        '.vrt'    : 'VRT',
        '.mem'    : 'MEMORY',
        '.sqlite' : 'SQLite',
        '.gdb'    : 'FileGDB',
        # Raster files
        '.tif'    : 'GTiff',
        '.ecw'    : 'ECW',
        '.mpr'    : 'ILWIS',
        '.mpl'    : 'ILWIS',
        '.jpg'    : 'JPEG',
        '.nc'     : 'netCDF',
        '.png'    : 'PNG',
        '.vrt'    : 'VRT',
        '.asc'    : 'AAIGrid',
        # Vector or Raster
        '.gpkg'   : 'GPKG'
    }
    
    return str(drv[os.path.splitext(_file)[1]])

In [ ]:
def get_shp_sref(shp):
    """
    Get Spatial Reference Object from Feature Class/Lyr
    """
    
    if type(shp) == ogr.Layer:
        lyr = shp
        
        c = 0
    
    else:
        data = ogr.GetDriverByName(
            drv_name(shp)).Open(shp)
        
        lyr = data.GetLayer()
        c = 1
    
    spref = lyr.GetSpatialRef()
    
    if c:
        del lyr
        data.Destroy()
    
    return spref

In [ ]:
# Open Dataset

pntdf = gp.read_file(pntshp)

In [ ]:
# Open Input
pnt_data = ogr.GetDriverByName(drv_name(pntshp)).Open(pntshp)
    
pnt_lyr = pnt_data.GetLayer()

# Get SRS for the output
srs = get_shp_sref(pnt_lyr)

# Create output
lnh_data = ogr.GetDriverByName(
    drv_name(lnhshp)).CreateDataSource(lnhshp)
    
lnh_lyr = lnh_data.CreateLayer(
    os.path.splitext(os.path.basename(lnhshp))[0],
    srs, geom_type=ogr.wkbLineString
)

# Copy fields from input to output
in_defn = pnt_lyr.GetLayerDefn()

field_names = []
for i in range(0, in_defn.GetFieldCount()):
    fdefn = in_defn.GetFieldDefn(i)
    name = fdefn.name
    
    if name != fid_field and name != pnt_order:
        lnh_lyr.CreateField(fdefn)
        field_names.append(name)

# Write data
feats = pntdf[fid_field].unique()

In [ ]:
lnh_lyr_defn = lnh_lyr.GetLayerDefn()
for feat in feats:
    fdf = pntdf[pntdf[fid_field] == feat]
    
    fdf.sort_values(by=[pnt_order], inplace=True)
    
    # Create line geometry
    geom = ogr.Geometry(ogr.wkbLineString)
    
    # Create new feature
    n_feat = ogr.Feature(lnh_lyr_defn)
    
    i = 0
    for idx, pnt in fdf.iterrows():
        pnt_geom = ogr.CreateGeometryFromWkt(pnt.geometry.wkt)
        
        geom.AddPoint(pnt_geom.GetX(), pnt_geom.GetY())
        
        if not i:
            for f in field_names:
                n_feat.SetField(f, pnt[f])
            
            i += 1
    
    n_feat.SetGeometry(geom)
            
    lnh_lyr.CreateFeature(n_feat)
            
    n_feat.Destroy()

del lnh_lyr
del pnt_lyr

pnt_data.Destroy()
lnh_data.Destroy()